In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import random
import torchaudio

random.seed(0)
np.random.seed(0)

In [2]:
!nvidia-smi

Tue Jun 13 16:58:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         Off| 00000000:01:00.0  On |                  N/A |
| 80%   59C    P0              154W / 350W|    668MiB / 24576MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load GoEmotions and General Audio Datasets (CREMA, TESS,  RAVDASS, ETC)

In [3]:
train_audio = pkl.load(open('./data/c4ai_clip/train_audio.pkl', "rb"))[['path', 'label']]
test_audio = pkl.load(open('./data/c4ai_clip/test_audio.pkl', "rb"))[['path', 'label']]
train_text = pkl.load(open('./data/c4ai_clip/train_text.pkl', "rb"))[['text', 'grouped_label']]
test_text = pkl.load(open('./data/c4ai_clip/test_text.pkl', "rb"))[['text', 'grouped_label']]

In [4]:
go_emotions = pd.concat([train_text, test_text])

In [5]:
go_emotions.groupby("grouped_label").count()

,text
grouped_label,
anger,6039
disgust,664
fear,705
joy,19002
neutral,14429
sadness,2936
surprise,5062


In [6]:
anger = go_emotions[go_emotions["grouped_label"] == "anger"].sample(5000, replace=False, random_state=0)
disgust = go_emotions[go_emotions["grouped_label"] == "disgust"].sample(4000, replace=True, random_state=0)
fear = go_emotions[go_emotions["grouped_label"] == "fear"].sample(4000, replace=True, random_state=0)
joy = go_emotions[go_emotions["grouped_label"] == "joy"].sample(5000, replace=False, random_state=0)
neutral = go_emotions[go_emotions["grouped_label"] == "neutral"].sample(5000, replace=False, random_state=0)
sadness = go_emotions[go_emotions["grouped_label"] == "sadness"].sample(2000, replace=True, random_state=0)

In [7]:
go_emotions = go_emotions[go_emotions["grouped_label"] != "anger"]
go_emotions = go_emotions[go_emotions["grouped_label"] != "joy"]
go_emotions = go_emotions[go_emotions["grouped_label"] != "neutral"]
go_emotions = pd.concat([go_emotions, anger, disgust, fear, joy, neutral, sadness])
go_emotions.groupby("grouped_label").count()

,text
grouped_label,
anger,5000
disgust,4664
fear,4705
joy,5000
neutral,5000
sadness,4936
surprise,5062


In [8]:
def norm_labels(x):
    if x == "afraid":
        return "fear"
    elif x == "angry":
        return "anger"
    elif x == "disgusted":
        return "disgust"
    elif x == "sad":
        return "sadness"
    else:
        return x

In [9]:
train_audio["label"] = train_audio["label"].apply(norm_labels)
test_audio["label"] = test_audio["label"].apply(norm_labels)

In [10]:
pd.concat([train_audio,test_audio]).groupby("label").count()

,path
label,
anger,1863
disgust,1863
fear,1863
joy,2055
neutral,1583
sadness,1863
surprise,592


In [11]:
audio_datasets = pd.concat([train_audio,test_audio]).reset_index(drop=True)
surprise = audio_datasets[audio_datasets["label"] == "surprise"].sample(2000, replace=True, random_state=0)
audio_datasets = audio_datasets[audio_datasets["label"] != "surprise"]
audio_datasets = pd.concat([audio_datasets, surprise]).reset_index(drop=True)
audio_datasets

,path,label
0,./audio/audio_emo/crema.woman.happy.14.wav,joy
1,./audio/audio_emo/crema.man.afraid.341.wav,fear
2,./audio/audio_emo/crema.man.neutral.370.wav,neutral
3,./audio/audio_emo/tess.woman.sad.350.wav,sadness
4,./audio/audio_emo/crema.man.angry.448.wav,anger
...,...,...
13085,./audio/audio_emo/tess.woman.surprised.351.wav,surprise
13086,./audio/audio_emo/ravdass.man.surprise.63.wav,surprise
13087,./audio/audio_emo/tess.woman.surprised.26.wav,surprise
13088,./audio/audio_emo/tess.woman.surprised.67.wav,surprise


In [12]:
audio_datasets.groupby("label").count()

,path
label,
anger,1863
disgust,1863
fear,1863
joy,2055
neutral,1583
sadness,1863
surprise,2000


## Load Meld and IEMOCAP

In [13]:
train_df_erc = pd.read_csv("train_text_df.csv", index_col=0).rename(columns={"utterance":"text"})
train_df_erc["path"] = train_df_erc["path"].apply(lambda x: '/home/vmachado/Documents/' + x)
train_df_erc

,text,label,path
0,The only one I know still love his parents. [B...,joy,/home/vmachado/Documents/multimodal-datasets/I...
1,The only one I know still love his parents. Ye...,neutral,/home/vmachado/Documents/multimodal-datasets/I...
2,Oh it's not bad thing it's good thing. You kno...,joy,/home/vmachado/Documents/multimodal-datasets/I...
3,"You know it's nice here, the air is sweet. You...",sadness,/home/vmachado/Documents/multimodal-datasets/I...
4,"You're not sorry you came? Not sorry, no. I c...",sadness,/home/vmachado/Documents/multimodal-datasets/I...
...,...,...,...
13723,That would be no. Come on. It doesn't taste ba...,neutral,/home/vmachado/Documents/multimodal-datasets/M...
13724,"Come on. It doesn't taste bad. Yeah, it's kind...",joy,/home/vmachado/Documents/multimodal-datasets/M...
13725,"Yeah, it's kinda sweet, sorta like, uh... Cant...",neutral,/home/vmachado/Documents/multimodal-datasets/M...
13726,Cantaloupe juice. Exactly. [BFR] You've tasted...,surprise,/home/vmachado/Documents/multimodal-datasets/M...


In [14]:
test_df_erc = pd.read_csv("test_text_df.csv", index_col=0).rename(columns={"utterance":"text"})
test_df_erc["path"] = test_df_erc["path"].apply(lambda x: '/home/vmachado/Documents/' + x)
test_df_erc

,text,label,path
0,"[BFR] Brian, I need help. [AFT] Babe, I don't...",sadness,/home/vmachado/Documents/multimodal-datasets/I...
1,"Brian, I need help. [BFR] Babe, I don't know w...",neutral,/home/vmachado/Documents/multimodal-datasets/I...
2,"Babe, I don't know what to tell you. Don't gi...",neutral,/home/vmachado/Documents/multimodal-datasets/I...
3,"I wish I had some answers for you, babe. I me...",neutral,/home/vmachado/Documents/multimodal-datasets/I...
4,I went to school and I got my degree. And I g...,neutral,/home/vmachado/Documents/multimodal-datasets/I...
...,...,...,...
3846,"Oh, it is. It isn't. [BFR] It is. [AFT] Isn't!",neutral,/home/vmachado/Documents/multimodal-datasets/M...
3847,It isn't. It is. [BFR] Isn't! [AFT],anger,/home/vmachado/Documents/multimodal-datasets/M...
3848,[BFR] Yeah baby! [AFT] I’m really glad you gu...,joy,/home/vmachado/Documents/multimodal-datasets/M...
3849,Yeah baby! [BFR] I’m really glad you guys are ...,neutral,/home/vmachado/Documents/multimodal-datasets/M...


In [15]:
test_df_erc["source"] = test_df_erc["path"].apply(lambda x: "meld" if "MELD" in x else "iemocap")
test_df_erc

,text,label,path,source
0,"[BFR] Brian, I need help. [AFT] Babe, I don't...",sadness,/home/vmachado/Documents/multimodal-datasets/I...,iemocap
1,"Brian, I need help. [BFR] Babe, I don't know w...",neutral,/home/vmachado/Documents/multimodal-datasets/I...,iemocap
2,"Babe, I don't know what to tell you. Don't gi...",neutral,/home/vmachado/Documents/multimodal-datasets/I...,iemocap
3,"I wish I had some answers for you, babe. I me...",neutral,/home/vmachado/Documents/multimodal-datasets/I...,iemocap
4,I went to school and I got my degree. And I g...,neutral,/home/vmachado/Documents/multimodal-datasets/I...,iemocap
...,...,...,...,...
3846,"Oh, it is. It isn't. [BFR] It is. [AFT] Isn't!",neutral,/home/vmachado/Documents/multimodal-datasets/M...,meld
3847,It isn't. It is. [BFR] Isn't! [AFT],anger,/home/vmachado/Documents/multimodal-datasets/M...,meld
3848,[BFR] Yeah baby! [AFT] I’m really glad you gu...,joy,/home/vmachado/Documents/multimodal-datasets/M...,meld
3849,Yeah baby! [BFR] I’m really glad you guys are ...,neutral,/home/vmachado/Documents/multimodal-datasets/M...,meld


In [16]:
test_df_erc.groupby("source").count()

,text,label,path
source,,,
iemocap,1241,1241,1241
meld,2610,2610,2610


## Join datasets

In [17]:
train_df_erc.groupby("label").count()

,text,path
label,,
anger,1954,1954
disgust,258,258
fear,266,266
joy,2783,2783
neutral,5804,5804
sadness,1451,1451
surprise,1212,1212


In [18]:
ang = train_df_erc[train_df_erc["label"] == "anger"].sample(3000, replace=True, random_state=0)
disg = train_df_erc[train_df_erc["label"] == "disgust"].sample(4700, replace=True, random_state=0)
fear = train_df_erc[train_df_erc["label"] == "fear"].sample(4700, replace=True, random_state=0)
joy = train_df_erc[train_df_erc["label"] == "joy"].sample(2300, replace=True, random_state=0)
sadness = train_df_erc[train_df_erc["label"] == "sadness"].sample(3500, replace=True, random_state=0)
surprise = train_df_erc[train_df_erc["label"] == "surprise"].sample(3800, replace=True, random_state=0)

In [19]:
train_df_erc_resampled = pd.concat([train_df_erc, joy, ang, disg, fear, surprise, sadness]).reset_index(drop=True)

In [20]:
train_df_erc_resampled.groupby("label").count()

,text,path
label,,
anger,4954,4954
disgust,4958,4958
fear,4966,4966
joy,5083,5083
neutral,5804,5804
sadness,4951,4951
surprise,5012,5012


## VoxPopuli + VoxCeleb

In [21]:
df_vox = pd.read_csv("voxceleb.csv").drop(columns="Unnamed: 0")[["path", "text", "sentiment_label"]]
df_vox

,path,text,sentiment_label
0,/home/vmachado/.cache/huggingface/datasets/dow...,and i i don't believe in god no religion says ...,Neutral
1,/home/vmachado/.cache/huggingface/datasets/dow...,the question because of my mother till i was f...,Neutral
2,/home/vmachado/.cache/huggingface/datasets/dow...,from my own culture things changed i i think a...,Neutral
3,/home/vmachado/.cache/huggingface/datasets/dow...,of god what is a creator the almighty that uh,Neutral
4,/home/vmachado/.cache/huggingface/datasets/dow...,i don't wanna pinpoint what exactly god is i i...,Neutral
...,...,...,...
7161,/home/vmachado/.cache/huggingface/datasets/dow...,the movie while he's solving this mystery exce...,Neutral
7162,/home/vmachado/.cache/huggingface/datasets/dow...,in my backstory you know that i actually uh hi...,Neutral
7163,/home/vmachado/.cache/huggingface/datasets/dow...,and it's just high action uh uh you want you,Neutral
7164,/home/vmachado/.cache/huggingface/datasets/dow...,you you can't stop thinking and and wondering ...,Neutral


In [22]:
df_ls = pd.read_csv("voxceleb.csv").drop(columns="Unnamed: 0")[["path", "text", "sentiment_label"]] #pd.read_csv("df_ls.csv")
df_ls

,path,text,sentiment_label
0,/home/vmachado/.cache/huggingface/datasets/dow...,and i i don't believe in god no religion says ...,Neutral
1,/home/vmachado/.cache/huggingface/datasets/dow...,the question because of my mother till i was f...,Neutral
2,/home/vmachado/.cache/huggingface/datasets/dow...,from my own culture things changed i i think a...,Neutral
3,/home/vmachado/.cache/huggingface/datasets/dow...,of god what is a creator the almighty that uh,Neutral
4,/home/vmachado/.cache/huggingface/datasets/dow...,i don't wanna pinpoint what exactly god is i i...,Neutral
...,...,...,...
7161,/home/vmachado/.cache/huggingface/datasets/dow...,the movie while he's solving this mystery exce...,Neutral
7162,/home/vmachado/.cache/huggingface/datasets/dow...,in my backstory you know that i actually uh hi...,Neutral
7163,/home/vmachado/.cache/huggingface/datasets/dow...,and it's just high action uh uh you want you,Neutral
7164,/home/vmachado/.cache/huggingface/datasets/dow...,you you can't stop thinking and and wondering ...,Neutral


In [23]:
#from joblib import Parallel, delayed
#_ = Parallel(n_jobs=12)(delayed(get_data_cluster)(x) for x in df_ls["path"])

## Join all datasets

In [24]:
df_train = pd.concat([go_emotions.rename(columns={"grouped_label":"label"}).assign(path=[None for _ in range(len(go_emotions))]), audio_datasets.assign(text=[None for _ in range(len(audio_datasets))]), train_df_erc_resampled, df_ls]).reset_index(drop=True) #.drop(columns="path")
#df_train = pd.concat([audio_datasets.assign(text=[None for _ in range(len(audio_datasets))]), train_df_erc_resampled,train_df_erc_resampled, df_ls]).reset_index(drop=True) #.drop(columns="path")

df_train

,text,label,path,sentiment_label
0,To make her feel threatened,fear,None,NaN
1,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,surprise,None,NaN
2,Demographics? I don’t know anybody under 35 wh...,surprise,None,NaN
3,Maybe that’s what happened to the great white ...,surprise,None,NaN
4,"I never thought it was at the same moment, but...",surprise,None,NaN
...,...,...,...,...
90346,the movie while he's solving this mystery exce...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90347,in my backstory you know that i actually uh hi...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90348,and it's just high action uh uh you want you,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90349,you you can't stop thinking and and wondering ...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral


In [25]:
import math

def label_to_sentiment(x):
    if x == None:
        return x
    if x in ["joy", "surprise"]:
        return "Positive"
    elif x in ["fear", "anger", "disgust", "sadness"]:
        return "Negative"
    else:
        return "Neutral"

In [26]:
df_train["sentiment_label"] = df_train["label"].apply(label_to_sentiment)

In [27]:
df_train

,text,label,path,sentiment_label
0,To make her feel threatened,fear,None,Negative
1,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,surprise,None,Positive
2,Demographics? I don’t know anybody under 35 wh...,surprise,None,Positive
3,Maybe that’s what happened to the great white ...,surprise,None,Positive
4,"I never thought it was at the same moment, but...",surprise,None,Positive
...,...,...,...,...
90346,the movie while he's solving this mystery exce...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90347,in my backstory you know that i actually uh hi...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90348,and it's just high action uh uh you want you,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral
90349,you you can't stop thinking and and wondering ...,NaN,/home/vmachado/.cache/huggingface/datasets/dow...,Neutral


In [28]:
df_train["sentiment_label"].value_counts()

Negative    46586
Positive    24212
Neutral     19553
Name: sentiment_label, dtype: int64

In [29]:
from sklearn.preprocessing import LabelEncoder

lab_encoder = LabelEncoder()
lab_encoder.fit(df_train['label'].unique())

lab_encoder_senti = LabelEncoder()
lab_encoder_senti.fit(df_train['sentiment_label'].unique())

LabelEncoder()

In [30]:
len(df_train)

90351

In [31]:
len(test_df_erc)

3851

In [32]:
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm

In [33]:
import torch
torch.manual_seed(0)
import torch.nn as nn

class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.2, contrast_mode='all',
                 base_temperature=0.2):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None, temperature=None, base_temperature=None, weights=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        if temperature == None:
            temperature = self.temperature
        if base_temperature == None:
            base_temperature = self.base_temperature
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (temperature/base_temperature) * mean_log_prob_pos
        
        if anchor_count == 1:
            loss = loss.view(anchor_count, batch_size)
        else:
            assert anchor_count == 2 and weights != None
            loss = loss.view(anchor_count * batch_size)
            loss = loss * weights
        return loss.mean()

In [34]:
from transformers import AutoTokenizer, AutoModel

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class TextEncoder(nn.Module):

    def __init__(self, model_name, max_len):
        super(TextEncoder, self).__init__()

        self.max_len = max_len
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        _ = self.tokenizer.add_tokens(['[NAME]', '[RELIGION]', '[LAUGHTER]', '[BFR]', '[AFT]'], special_tokens=True)
        self.encoder = AutoModel.from_pretrained(model_name)
        self.encoder.resize_token_embeddings(len(self.tokenizer))
 
    def forward(self, sentences):
        
        tokenized = self.tokenizer(sentences, padding='max_length', truncation=True, return_tensors='pt', max_length=self.max_len)
        
        att_mask = tokenized["attention_mask"].to(0)
        tokenized = {
            "input_ids":tokenized["input_ids"].to(0),
            "attention_mask": att_mask
        }
        out = self.encoder(**tokenized)
        
        out = mean_pooling(out, att_mask)
        #out = out[:, 0, :]
        return out

## MFCC Extractor and KMeans Hidden units

In [35]:
import hashlib
import torch
import librosa
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torchaudio
import numpy as np
from tqdm import tqdm
import functools
import math
import pickle as pkl
import os

sample_rate = 16000
n_fft = 400.0
frame_length = n_fft / sample_rate * 1000.0
frame_shift = frame_length / 2.0

MFCC_PARAMS = {
    "channel": 0,
    "dither": 0.0,
    "window_type": "hanning",
    "frame_length": frame_length,
    "frame_shift": frame_shift,
    "remove_dc_offset": False,
    "round_to_power_of_two": False,
    "sample_frequency": sample_rate,
}

PITCH_PARAMS = {
    "frame_length": frame_length,
    "frame_shift": frame_shift,
    "sample_rate": sample_rate,
}

def get_feats(x, sr, max_pool_window_size=4, params=MFCC_PARAMS, params_pitch=PITCH_PARAMS):
    pitch = torchaudio.functional.compute_kaldi_pitch(x, **params_pitch).squeeze(dim=0)
    
    x = x.view(1, -1)

    mfccs = torchaudio.compliance.kaldi.mfcc(
        x,
        **params
    )  # (time, freq)
    
    try:
        mfccs = torch.cat([mfccs, pitch], dim=-1)
    except:
        mfccs = torch.cat([mfccs, torch.Tensor(np.zeros((mfccs.shape[0], 2)))], dim=-1)
    
    mfccs_z = torch.Tensor(np.zeros(((mfccs.shape[0] // max_pool_window_size) + 1, mfccs.shape[1])))
    
    for i in range(len(mfccs) // max_pool_window_size): # Max pooling over time to reduce sequence size
        mfcc_win = mfccs[i * max_pool_window_size:(i + 1) * max_pool_window_size]
        norms = [np.linalg.norm(v[:-2]) for v in mfcc_win]
        argmax = np.argmax(np.array(norms))
        mfccs_z[i] = mfcc_win[argmax]
            
    mfccs = mfccs_z.transpose(0, 1)  # (freq, time)
    deltas = torchaudio.functional.compute_deltas(mfccs)
    ddeltas = torchaudio.functional.compute_deltas(deltas)
    concat = torch.cat([mfccs, deltas, ddeltas], dim=0)
    concat = concat.transpose(0, 1).contiguous()
    
    return concat

def mfcc_feature_extractor(path, desired_sr=16000, cache="./preprocessed_audio_cache_new"):
    hashed_name = hashlib.md5(path.encode('utf-8')).hexdigest()
    hashed_path = cache + '/' + f"{hashed_name}.bin"
    with torch.no_grad():
        if os.path.isfile(hashed_path):
            return torch.load(hashed_path)
        else:
            waveform, sample_rate = torchaudio.load(path, normalize=True, channels_first=True)
            waveform = waveform.float()
                
            if len(waveform.shape) == 2:
                waveform = torch.mean(waveform, dim=0).unsqueeze(dim=0)

            if sample_rate != desired_sr:
                transform = torchaudio.transforms.Resample(sample_rate, desired_sr)
                waveform = transform(waveform)

            mfcc = get_feats(waveform, desired_sr)
            torch.save(mfcc, hashed_path)
            return mfcc

In [36]:
def get_data_cluster(path):
    mfcc_audio = mfcc_feature_extractor(path)
    return mfcc_audio

In [37]:
get_data_cluster("./audio/audio_emo/tess.woman.sad.5.wav")

tensor([[-5.4309e+01, -3.1988e+01, -7.8407e+00,  ...,  5.5061e-01,
          3.8081e-03, -4.5475e-14],
        [-4.4171e+01, -3.9444e+01,  5.1471e+00,  ...,  4.6405e-01,
         -3.2662e-04, -4.5475e-14],
        [-3.8823e+01, -5.1685e+01,  8.3763e+00,  ..., -6.5955e-01,
          1.7061e-02, -4.5475e-14],
        ...,
        [-6.4814e+01, -1.1119e+01,  1.3244e+01,  ..., -9.4502e-01,
         -4.3116e-02, -1.9203e+01],
        [-6.6382e+01, -1.6293e+01,  1.4263e+01,  ..., -2.1900e-01,
         -4.5776e-02, -1.4936e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  5.0201e-01,
         -9.7358e-03, -4.2674e+00]])

## Add mask to Transformer, try learned positional embeddings

In [38]:
from joblib import Parallel, delayed

class AudioEncoderMFCCHUTokenizer(object):
    
    def __init__(self, max_length, mean, std):
        self.max_length = max_length
        self.mean = mean
        self.std = std
        
    def tokenize(self, path):
        with torch.no_grad():
            mfcc = (mfcc_feature_extractor(path) - self.mean) / (self.std + 1e-10)
        
            l = len(mfcc)
            att_mask = torch.ones((self.max_length, 1))

            if l > self.max_length:
                mfcc = mfcc[:self.max_length]
            elif l < self.max_length:
                mask_idx = torch.Tensor([i + l for i in range(self.max_length - l)]).long()
                att_mask = att_mask.index_fill_(0, mask_idx, 0.0)
                repeat = torch.zeros((self.max_length - l, mfcc.shape[1]))
                mfcc = torch.cat([mfcc, repeat], dim=0)

            mfcc = mfcc.unsqueeze(dim=0)
            att_mask = att_mask.unsqueeze(dim=0).squeeze(dim=-1)
            return mfcc, att_mask
    
    def batch_tokenize(self, paths, n_jobs=12):
        X = Parallel(n_jobs=n_jobs)(delayed(self.tokenize)(x) for x in paths)
        mfccs = [m for m, _ in X]
        att_masks = [a for _, a in X]
        mfccs = torch.cat(mfccs, dim=0)
        att_masks = torch.cat(att_masks, dim=0)
        return mfccs, att_masks

In [39]:
from vector_quantize_pytorch import VectorQuantize, GroupedResidualVQ
from vector_quantize_pytorch import ResidualVQ

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class AudioEncoderMFCCHU(nn.Module):

    def __init__(self, 
                 vocab_size,
                 emb_dim=768, 
                 n_layers=6, 
                 max_length=800,
                 raw_features_size=45,
                 nheads=8, 
                 dropout=0.2):
        super(AudioEncoderMFCCHU, self).__init__()
        
        self.vocab_size = vocab_size
        
        self.max_length = max_length
        
        self.vq = ResidualVQ(
            dim = raw_features_size,
            codebook_size = self.vocab_size,
            codebook_dim = 128,
            num_quantizers = 4,
            threshold_ema_dead_code = 2,
        )
            
        self.pos_encoder = PositionalEncoding(emb_dim, dropout=0.0)
        self.project = nn.Sequential(nn.Linear(raw_features_size, emb_dim), nn.GELU(), nn.Linear(emb_dim, emb_dim))
        self.emb_dim = emb_dim
        self.dropout = dropout
        self.transf_layer = nn.TransformerEncoderLayer(d_model=emb_dim, dim_feedforward=emb_dim*4, nhead=nheads, batch_first=True, norm_first=True, dropout=self.dropout)
        self.transf_enc = nn.TransformerEncoder(self.transf_layer, num_layers=n_layers, norm=None)
        
        self.norm_feats = nn.LayerNorm(raw_features_size)

    def forward(self, features, attn_masks):
        
        features = self.norm_feats(features)
        qtz_feats, _, vq_loss = self.vq(features)
        vq_loss = vq_loss.mean()
            
        seq_lens = 1 / torch.sum(attn_masks, dim=-1)
        seq_lens = seq_lens.unsqueeze(dim=-1)

        x = self.project(qtz_feats)
        x = self.pos_encoder(x)

        x = self.transf_enc(x, src_key_padding_mask=attn_masks)
        x = seq_lens * torch.sum(x, dim=1)
        
        return x, vq_loss

In [40]:
class AudioTextCLIP(nn.Module):

    def __init__(self, 
                 text_encoder, 
                 audio_encoder, 
                 freeze_text_enc=False, 
                 freeze_audio_enc=False, 
                 in_features_text=384, 
                 in_features_audio=16, 
                 wide_proj=1024, 
                 proj_size=128,
                 hidden_size=384,
                 rate=0.1,):
        super(AudioTextCLIP, self).__init__()
        
        self.audio_encoder = audio_encoder
        self.text_encoder = text_encoder

        if freeze_text_enc:
            for i, (name, param) in enumerate(list(self.text_encoder.named_parameters())):
                param.requires_grad = False
            
        if freeze_audio_enc:
            for i, (name, param) in enumerate(list(self.audio_encoder.named_parameters())):
                param.requires_grad = False

        self.mods_proj = nn.Sequential(nn.Linear(hidden_size, hidden_size), nn.GELU(), nn.Dropout(p=rate), nn.Linear(hidden_size, wide_proj))
        self.text_proj = nn.Sequential(self.text_encoder, nn.Linear(in_features_text, hidden_size),  nn.GELU(), nn.Dropout(p=rate), nn.Linear(hidden_size, hidden_size), nn.GELU(), self.mods_proj)
        self.audio_proj = nn.Sequential(nn.Linear(in_features_audio, hidden_size), nn.GELU(), nn.Dropout(p=rate), nn.Linear(hidden_size, hidden_size), nn.GELU(), self.mods_proj)
        self.linear = nn.Linear(wide_proj, proj_size)
        
        self.alpha = nn.Sequential(nn.Linear(wide_proj, wide_proj), nn.Dropout(p=rate), nn.Linear(wide_proj, 1))
        
        self.rate = rate
        
    def forward(self, inp):

        sentences, audio_input, multimodal = inp
        
        assert sentences != None or audio_input != None or multimodal != None
        
        x_text = None
        x_text_wide = None
        if sentences != None:
            x_text_wide = F.normalize(self.text_proj(sentences), dim=-1)
            x_text = F.normalize(self.linear(x_text_wide), dim=-1)
            
        x_audio = None
        x_audio_wide = None
        if audio_input != None:
            x_audio_wide, _ = self.audio_encoder(**audio_input)
            x_audio_wide = F.normalize(self.audio_proj(x_audio_wide), dim=-1)
            x_audio = F.normalize(self.linear(x_audio_wide), dim=-1)
        
        x_mult_text = None
        x_mult_text_wide = None
        x_mult_audio = None
        x_mult_audio_wide = None
        
        # Approximate text and audio, and make sum of vectors point to correct cls
        if multimodal != None:
            x_mult_text = F.normalize(self.text_proj(multimodal['sentences']), dim=-1)
            x_mult_text_alpha = self.alpha(x_mult_text)
            
            x_mult_audio, _ = self.audio_encoder(**multimodal['audio_input'])
            x_mult_audio = F.normalize(self.audio_proj(x_mult_audio), dim=-1)
            x_mult_audio_alpha = self.alpha(x_mult_audio)
            
            alphas = F.softmax(torch.cat([x_mult_text_alpha, x_mult_audio_alpha], dim=-1), dim=-1).unsqueeze(dim=1)

            # View 1
            x_mult_text_wide = alphas[:, :, 1] * x_mult_text
            x_mult_text = alphas[:, :, 1] * F.normalize(self.linear(x_mult_text), dim=-1)
            
            # View 2
            x_mult_audio_wide = alphas[:, :, 0] * x_mult_audio
            x_mult_audio = alphas[:, :, 0] * F.normalize(self.linear(x_mult_audio), dim=-1)

        return {
            "x_text": x_text,
            "x_text_wide": x_text_wide,
            "x_audio": x_audio,
            "x_audio_wide": x_audio_wide,
            "x_mult_text": x_mult_text,
            "x_mult_text_wide": x_mult_text_wide,
            "x_mult_audio": x_mult_audio,
            "x_mult_audio_wide": x_mult_audio_wide,
        }

In [41]:
df_train['label'].unique()

array(['fear', 'surprise', 'sadness', 'disgust', 'anger', 'joy',
       'neutral', nan], dtype=object)

In [42]:
import torchaudio
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
gc.enable()

In [43]:
gc.collect()

0

In [44]:
import numpy as np
import faiss

class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

In [45]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler

class Scheduler(_LRScheduler):
    def __init__(self, 
                 optimizer: Optimizer,
                 dim_embed: int,
                 warmup_steps: int,
                 last_epoch: int=-1,
                 verbose: bool=False) -> None:

        self.dim_embed = dim_embed
        self.warmup_steps = warmup_steps
        self.num_param_groups = len(optimizer.param_groups)

        super().__init__(optimizer, last_epoch, verbose)
        
    def get_lr(self) -> float:
        lr = calc_lr(self._step_count, self.dim_embed, self.warmup_steps)
        return [lr] * self.num_param_groups

global PREVIOUS_LR
PREVIOUS_LR = -9999
def calc_lr(step, dim_embed, warmup_steps):
    #if step > warmup_steps:
    #    return 5e-5
    global PREVIOUS_LR
    lr = dim_embed**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))
    #return lr
    if lr < 1e-4:
        PREVIOUS_LR = lr
        return lr
    else:
        #lr = dim_embed**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))
        while lr >= PREVIOUS_LR:
            step += 1.
            lr = dim_embed**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))
        PREVIOUS_LR = lr
        return lr

In [46]:
dim_embed = 768
N_VECTORS = 100
MAX_LEN = 128

audio_encoder = AudioEncoderMFCCHU(
    N_VECTORS, 
    emb_dim=dim_embed, 
    n_layers=3, 
    max_length=MAX_LEN, 
    nheads=8,
    dropout=0.2
)

#audio_encoder = torch.load(f'audio_encoder_best/audio_encoder.bin')
#audio_encoder.load_state_dict(torch.load(f'audio_encoder_pre_trained_reformed_5_FIM_6_layer_continue/audio_best.bin'))

In [47]:
#MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
text_encoder = TextEncoder(MODEL_NAME, max_len=128)
#text_encoder.load_state_dict(torch.load('text_encoder_only_meld/dabest_text.bin'))
#text_encoder.load_state_dict(torch.load(f'text_encoder_ready_L2_test2/best_text_encoder.bin'))
#text_encoder.load_state_dict(torch.load(f'text_encoder_ready_L2_test2/pytorch_model_AudioTextCLIP_epoch_22.bin'))

In [48]:
mean = torch.Tensor([-2.4002e+01, -5.4726e+00, -1.1825e+01,  6.6728e-01, -1.6782e+01,
        -7.0169e+00, -1.4942e+01, -7.5624e+00, -7.5826e+00, -2.1841e+00,
        -6.0805e+00, -3.0830e+00, -5.6635e+00,  1.9363e-01,  2.0110e+02,
         7.1090e-01,  2.1245e-01,  1.8659e-01,  1.7632e-02,  3.0055e-01,
         5.8593e-02,  2.2993e-01,  1.1659e-01,  1.2025e-01,  6.4304e-03,
         1.1272e-01,  5.9370e-02,  1.0354e-01, -7.5894e-03, -4.8167e+00,
         2.0838e-01,  5.0690e-02,  1.0683e-01,  1.4690e-02,  1.4356e-01,
         6.8523e-02,  1.3284e-01,  6.2631e-02,  6.9484e-02,  3.4108e-02,
         5.7172e-02,  4.1190e-02,  4.6194e-02, -1.4585e-03, -1.8164e+00])
std = torch.Tensor([1.4204e+01, 1.3486e+01, 1.7198e+01, 1.7307e+01, 1.7100e+01, 1.7898e+01,
        1.7963e+01, 1.7448e+01, 1.7087e+01, 1.7854e+01, 1.5724e+01, 1.5907e+01,
        1.4398e+01, 4.2155e-01, 9.8201e+01, 3.8302e+00, 4.1511e+00, 4.9767e+00,
        5.4117e+00, 5.3979e+00, 5.6093e+00, 5.7001e+00, 5.3550e+00, 5.4432e+00,
        5.3551e+00, 4.9909e+00, 4.8445e+00, 4.4496e+00, 1.3624e-01, 1.9747e+01,
        1.5212e+00, 1.8603e+00, 2.2115e+00, 2.4687e+00, 2.3968e+00, 2.5010e+00,
        2.5413e+00, 2.3874e+00, 2.4448e+00, 2.3907e+00, 2.2411e+00, 2.1623e+00,
        1.9933e+00, 6.0878e-02, 6.4246e+00])

audio_tokenizer = AudioEncoderMFCCHUTokenizer(MAX_LEN, mean, std)

In [49]:
gc.collect()

0

# Save adio encoder again

In [50]:
def sup_contrastive_loss(embeddings, targets, temperature=0.5):
    batch_size = embeddings.size(0)
    
    mask = torch.eye(batch_size, device=embeddings.device, dtype=torch.bool)
    
    similarity_matrix = torch.matmul(embeddings, embeddings.T) / temperature
    similarity_matrix = similarity_matrix.masked_fill(mask, 0.0)
    
    targets = targets.contiguous().view(-1, 1)
    targets = torch.eq(targets, targets.T).float()
    targets = targets.masked_fill(mask, 0.0)
    
    loss = - (targets * F.log_softmax(similarity_matrix, dim=-1)).sum(dim=-1) / targets.sum(dim=-1)
    
    return loss.mean()

In [51]:
def unsupervised_contrastive_loss(embeddings1, embeddings2, temperature=0.5, weights=None):
    assert embeddings1.size(0) == embeddings2.size(0)
    batch_size = embeddings1.size(0)
    embeddings = torch.cat([embeddings1, embeddings2], dim=0) # (2xN,f)

    mask = torch.eye(2 * batch_size, device=embeddings.device, dtype=torch.bool)
    
    similarity_matrix = torch.matmul(embeddings, embeddings.T) / temperature # (2xN,2xN)
    similarity_matrix = similarity_matrix.masked_fill(mask, 0.0)
    
    targets = torch.eye(batch_size, device=embeddings.device, dtype=torch.float32)
    targets = targets.repeat(2, 2) # (2xN, 2xN) -> [[I, I], [I, I]]
    targets = targets.masked_fill(mask, 0.0) # (2xN, 2xN) -> [[0, I], [I, 0]]
    
    loss = - (targets * F.log_softmax(similarity_matrix, dim=-1)).sum(dim=-1) 
    loss = loss * weights

    return loss.mean()

In [52]:
#torch.save(supcon_model.audio_encoder, f'audio_encoder_best/audio_encoder.bin')

In [53]:
#torch.norm(out["x_mult_text"] + out["x_mult_audio"], dim=-1)

In [54]:
sum(df_train["sentiment_label"].isna())

0

In [55]:
#mult = batch[batch["text"].notna()]
#mult = mult[mult["path"].notna()].reset_index(drop=True)
#mult["label"].value_counts()

In [56]:
#sup_contrastive_loss(out_x_lab, target, temperature=0.2)

In [57]:
#out_x_lab @ out_x_lab.T

In [ ]:
train_ds = torch.utils.data.TensorDataset(torch.Tensor(list(range(len(df_train)))))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=512, shuffle=True)

test_ds = torch.utils.data.TensorDataset(torch.Tensor(list(range(len(test_df_erc)))))
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1024, shuffle=False)

PATH_TO_SAVE = 'ESTAMOS_PERTO_AMIGO_ESTOU_AQUI_4_freezed_8'
!mkdir -p {PATH_TO_SAVE}

supcon_model = AudioTextCLIP(
    text_encoder,
    audio_encoder,
    in_features_text=384,
    in_features_audio=dim_embed, 
    hidden_size=1024,
    wide_proj=2048,
    proj_size=128, 
    freeze_text_enc=False,
    freeze_audio_enc=False,
    rate=0.2,
)

# Grid search best temperatures
# Try to only fine tune on evaluation datasets
#supcon_model.load_state_dict(torch.load(f'ESTAMOS_PERTO_AMIGO_ESTOU_AQUI_4_freezed_4_layer/pytorch_model_AudioTextCLIP_epoch_9.bin')['model'])

supcon_loss = SupConLoss(temperature=0.2, contrast_mode='all', base_temperature=0.2)
supcon_loss_senti = SupConLoss(temperature=0.1, contrast_mode='all', base_temperature=0.1)
supcon_loss_intra = SupConLoss(temperature=0.6, contrast_mode='all', base_temperature=0.6)

supcon_model.to(0)

scaler = torch.cuda.amp.GradScaler()

step = 0
e = 0
patience = 9999
early_stop_flag = 0
old_f1 = -float('inf')

param_optimizer = list(supcon_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{
    'params':
    [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay_rate':
    0.1
}, {
    'params':
    [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    'weight_decay_rate':
    0.0
}]

scheduler_epochs = 5
opt = torch.optim.AdamW(optimizer_grouped_parameters, lr=0, betas=(0.9, 0.98), eps=1e-8)
#scheduler = torch.optim.lr_scheduler.LinearLR(opt, start_factor=0.5, end_factor=0.9, total_iters=10, last_epoch=- 1, verbose=False)
scheduler = Scheduler(opt, 768, 600)

epochs = 9999

while e < epochs:
    supcon_model.train()
    epoch_loss = 0.0
    proj_val = []
    targets_val = []

    proj_train = []
    targets_train = []

    for i, batch_indices in enumerate(tqdm(train_loader, total=len(train_loader))):
        if i == len(train_loader)-1:
            continue
        batch = df_train.iloc[batch_indices[0]]
        only_text = batch[batch["path"].isna()]
        sentences = only_text["text"].tolist()
        y_text = torch.Tensor(lab_encoder.transform(only_text["label"]))
        y_text_senti = torch.Tensor(lab_encoder_senti.transform(only_text["sentiment_label"]))
        
        only_audio = batch[batch["text"].isna()]
        audio_paths = only_audio["path"].tolist()
        try:
            mfccs, att = audio_tokenizer.batch_tokenize(audio_paths)

            audio_input = {
                "features": mfccs.to(0),
                "attn_masks": att.to(0),
            }
        except:
            audio_input = None

        y_audio = torch.Tensor(lab_encoder.transform(only_audio["label"]))
        y_audio_senti = torch.Tensor(lab_encoder_senti.transform(only_audio["sentiment_label"]))
        
        mult = batch[batch["text"].notna()]
        mult = mult[mult["path"].notna()].reset_index(drop=True)
        
        mult_not_na_idx = mult[mult["label"].notna()].index
        #batch_not_na_idx = batch[batch["label"].notna()].index
        #mult_na_idx = mult[mult["label"].isna()].index
        
        y_mult = torch.Tensor(lab_encoder.transform(mult.iloc[mult_not_na_idx]["label"]))
        
        y_mult_senti = torch.Tensor(lab_encoder_senti.transform(mult["sentiment_label"]))
        
        audio_path_mult = [str(t['path']) for _, t in mult.iterrows()]
        
        mfccs_mult, att_mult = audio_tokenizer.batch_tokenize(audio_path_mult)
        
        sentences_mult = [str(t['text']) for _, t in mult.iterrows()]
        
        multimodal = {'sentences': sentences_mult, 
                      'audio_input': {"features": mfccs_mult.to(0), "attn_masks": att_mult.to(0)}}
        
        target = torch.cat([y_text, y_audio, y_mult]).long().cuda()
        target_senti = torch.cat([y_text_senti, y_audio_senti, y_mult_senti]).long().cuda()
        
        x = [sentences, audio_input, multimodal]
        
        if len(sentences) == 0:
            x[0] = None
        if len(audio_paths) == 0:
            x[1] = None
        if len(sentences_mult) == 0:
            x[2] = None

        with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16) as autocast, torch.backends.cuda.sdp_kernel(enable_flash=False) as disable:
            
            out = supcon_model(x)
            
            # Multimodal loss
            if x[-1] is not None:
                x_mult_text = out["x_mult_text"]
                x_mult_audio = out["x_mult_audio"]
                x_mult_text_norm = F.normalize(x_mult_text, dim=-1)
                x_mult_audio_norm = F.normalize(x_mult_audio, dim=-1)
                
                #weights = torch.stack([torch.norm(x_mult_audio, dim=-1), torch.norm(x_mult_text, dim=-1)], dim=0).detach()
                weights = torch.cat([torch.norm(x_mult_audio, dim=-1), torch.norm(x_mult_text, dim=-1)], dim=0).detach()
                #weights = torch.ones_like(weights)
                
                # Augument modality
                augs = random.choices(
                    population=[0, 1, 2],
                    weights=[0.5, 0.25, 0.25],
                    k=len(x_mult_text)
                )
                
                x_mult = torch.stack([F.normalize(x_mult_text + x_mult_audio, dim=-1), 
                                      x_mult_text_norm, 
                                     x_mult_audio_norm], dim=1)

                x_mult = x_mult[list(range(len(augs))), augs, :] 
                x_mult_wide = F.normalize(out["x_mult_text_wide"] + out["x_mult_audio_wide"], dim=-1)
                
                # Add weighted contrastive loss
                #x_mult_text = x_mult_text_norm.unsqueeze(dim=1)
                #x_mult_audio = x_mult_audio_norm.unsqueeze(dim=1)
                #mult = torch.cat([x_mult_text, x_mult_audio], dim=1)
                
                out_x, out_x_wide = None, None
                
                if x[0] is not None:
                    if x[1] is not None:
                        out_x = torch.cat([out["x_text"], out["x_audio"], x_mult], dim=0) #.unsqueeze(dim=1)
                        out_x_lab = torch.cat([out["x_text"], out["x_audio"], x_mult[mult_not_na_idx]], dim=0) #.unsqueeze(dim=1)
                        out_x_wide = torch.cat([out["x_text_wide"], out["x_audio_wide"], x_mult_wide[mult_not_na_idx]], dim=0)
                    else:
                        out_x = torch.cat([out["x_text"], x_mult], dim=0) #.unsqueeze(dim=1)
                        out_x_lab = torch.cat([out["x_text"], x_mult[mult_not_na_idx]], dim=0) #.unsqueeze(dim=1)
                        out_x_wide = torch.cat([out["x_text_wide"], x_mult_wide[mult_not_na_idx]], dim=0)
                elif x[1] is not None:
                    out_x = torch.cat([out["x_audio"], x_mult], dim=0) #.unsqueeze(dim=1)
                    out_x_lab = torch.cat([out["x_audio"], x_mult[mult_not_na_idx]], dim=0) #.unsqueeze(dim=1)
                    out_x_wide = torch.cat([out["x_audio_wide"], x_mult_wide[mult_not_na_idx]], dim=0)
                else:
                    out_x = x_mult.unsqueeze(dim=1)
                    out_x_lab = x_mult[mult_not_na_idx] #.unsqueeze(dim=1)
                    out_x_wide = x_mult_wide[mult_not_na_idx]
                
                # fera ta
                
                loss = 0.8 * (0.5 * sup_contrastive_loss(out_x_lab, target, temperature=0.2) + 0.5 * sup_contrastive_loss(out_x, target_senti, temperature=0.1)) + 0.2 * unsupervised_contrastive_loss(x_mult_text_norm, x_mult_audio_norm, temperature=0.8, weights=weights)
                #loss = 0.5 * (0.5 * supcon_loss(out_x_lab, labels=target) + 0.5 * supcon_loss_senti(out_x, labels=target_senti)) + 0.5 * supcon_loss_intra(mult, weights=weights) 
            else:
                if x[0] is not None:
                    if x[1] is not None:
                        out_x = torch.cat([out["x_text"], out["x_audio"]], dim=0).unsqueeze(dim=1)
                        out_x_wide = torch.cat([out["x_text_wide"], out["x_audio_wide"]], dim=0)
                    else:
                        out_x = out["x_text"]
                        out_x_wide = out["x_text_wide"]
                else:
                    if x[1] is not None:
                        out_x = out["x_audio"]
                        out_x_wide = out["x_audio_wide"]
                    else:
                        raise Exception("Nothing to work :()")
                        
                loss = supcon_loss(out_x, labels=target) + supcon_loss(out_x, labels=target_senti)

        scaler.scale(loss).backward()
        scaler.unscale_(opt)

        torch.nn.utils.clip_grad_norm_(supcon_model.parameters(), 20.0)
        scaler.step(opt)
        scheduler.step()
        scaler.update()
        
        opt.zero_grad(set_to_none=True)

        epoch_loss += loss.item()
        proj_train.append(np.array(out_x_wide.detach().cpu()))
        targets_train.append(np.array(target.cpu()))
        if i == 20:
            print(loss.item())
            break
        del out_x
        del x_mult
        del out_x_wide
        gc.collect()
        torch.cuda.empty_cache()
    #scheduler.step()
    proj_train = np.concatenate(proj_train, axis=0)
    targets_train = np.concatenate(targets_train, axis=0)
    
    clf = FaissKNeighbors(k=128)
    clf.fit(proj_train, np.array(targets_train, dtype=int))

    epoch_loss = epoch_loss/len(train_loader)
    supcon_model.eval()
    preds = []
    targets = []
    css = 0.0
    wide_audio = []
    
    for i, batch_indices in enumerate(tqdm(test_loader, total=len(test_loader))):
        with torch.no_grad():
        
            multimodal_batch = test_df_erc.iloc[batch_indices[0]]

            audio_path_mult = [str(t['path']) for _, t in multimodal_batch.iterrows()]
            mfccs_mult, att_mult = audio_tokenizer.batch_tokenize(audio_path_mult)

            sentences_mult = [str(t['text']) for _, t in multimodal_batch.iterrows()]

            multimodal = {'sentences': sentences_mult, 
                          'audio_input': {"features": mfccs_mult.to(0), "attn_masks": att_mult.to(0)}}
        
            target = torch.Tensor(lab_encoder.transform(list(multimodal_batch["label"])))

            x = [None, None, multimodal]
            with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16) as autocast, torch.backends.cuda.sdp_kernel(enable_flash=False) as disable:
                out = supcon_model(x)
            
            # Multimodal loss
            out_x_wide = F.normalize(out["x_mult_text_wide"] + out["x_mult_audio_wide"], dim=-1)
            #out_x_wide = F.normalize(out["x_mult_audio_wide"], dim=-1)
            
            cs = F.cosine_similarity(F.normalize(out["x_mult_text_wide"], dim=-1), F.normalize(out["x_mult_audio_wide"], dim=-1))

            wide = np.array(out_x_wide.cpu())
            wide_audio.append(np.array(F.normalize(out["x_mult_audio_wide"], dim=-1).cpu()))
            pred = clf.predict(wide)
            preds.append(pred)

            assert len(wide) == len(pred)

            proj_val.append(wide)
            targets_val.append(np.array(target.cpu()))
            css += np.sum(np.array(cs.cpu()))
            del out_x_wide
            gc.collect()
            torch.cuda.empty_cache()

    proj_val = np.concatenate(proj_val, axis=0)
    wide_audio = np.concatenate(wide_audio, axis=0)
    targets_val = np.concatenate(targets_val, axis=0)
    
    preds = np.array(np.concatenate(preds, axis=0))
    
    css = css / len(test_df_erc)

    general_f1 = f1_score(targets_val, preds, average='weighted')
    general_acc = accuracy_score(targets_val, preds)
    
    print(f'Cosine Similarity between mods: {css}')
    
    meld_idx = test_df_erc[test_df_erc["source"] == "meld"].index
    iemocap_idx = test_df_erc[test_df_erc["source"] != "meld"].index
    
    general_f1_iemocap = f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='weighted')
    general_f1_iemocap_audio = f1_score(targets_val[iemocap_idx], clf.predict(wide_audio)[iemocap_idx], average='weighted')
    general_acc_iemocap = accuracy_score(targets_val[iemocap_idx], preds[iemocap_idx])
    
    general_f1_meld = f1_score(targets_val[meld_idx], preds[meld_idx], average='weighted')
    general_acc_meld = accuracy_score(targets_val[meld_idx], preds[meld_idx])
    
    print(f'General - KNN F1: {general_f1} Acc: {general_acc}')
    print(f'Iemocap - KNN F1: {general_f1_iemocap} Acc: {general_acc_iemocap}')
    print(f'Iemocap - KNN F1 - Only Audio: {general_f1_iemocap_audio}')
    print(f'Meld - KNN F1: {general_f1_meld} Acc: {general_acc_meld}')
    
    print(f"Iemocap - KNN F1 (macro): {f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='macro')}")
    print(f"Meld - KNN F1 (macro): {f1_score(targets_val[meld_idx], preds[meld_idx], average='macro')}")

    try:
        tsne = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=5).fit_transform(proj_val)

        sns.scatterplot(x=tsne[:, 0], y=tsne[:, 1], hue=lab_encoder.inverse_transform(list(np.array(targets_val, dtype=int))) , palette='tab10')
        plt.show()
    
    except:
        pass
    
    print(f'Epoch: {e + 1} - Train Loss: {epoch_loss}')
    e += 1
    
    #if e == scheduler_epochs: # Unfreeze text encoder
    #    for i, (name, param) in enumerate(list(supcon_model.text_encoder.named_parameters())):
    #        param.requires_grad = True

    with open(f"{PATH_TO_SAVE}/metrics_epoch_{e}.txt", "w") as f:
        f.write(f'General - KNN F1: {general_f1} Acc: {general_acc}')
        f.write(f'Iemocap - KNN F1: {general_f1_iemocap} Acc: {general_acc_iemocap}')
        f.write(f'Meld - KNN F1: {general_f1_meld} Acc: {general_acc_meld}')
        f.write(f"Iemocap - KNN F1 (macro): {f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='macro')}")
        f.write(f"Meld - KNN F1 (macro): {f1_score(targets_val[meld_idx], preds[meld_idx], average='macro')}")
        
    checkpoint = {"model": supcon_model.state_dict(),
              "optimizer": opt.state_dict(),
              "scaler": scaler.state_dict()}
    torch.save(checkpoint, f'{PATH_TO_SAVE}/pytorch_model_AudioTextCLIP_epoch_{e}.bin')

  0%|                                                   | 0/177 [00:00<?, ?it/s]/home/vmachado/anaconda3/envs/dl_ds_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  5%|██▏                                        | 9/177 [00:16<04:29,  1.61s/it]

In [ ]:
target.shape

In [ ]:
len(y_text)

In [ ]:
len(y_audio)

In [ ]:
len(y_mult)

In [ ]:
len(mult_not_na_idx)

In [ ]:
1608 + 3757

In [ ]:
out_x.shape

In [ ]:
out_x[mult_not_na_idx].shape

In [ ]:
xxx

In [ ]:
'''
Cosine Similarity between mods: -0.13657516783951593
General - KNN F1: 0.6451182550010343 Acc: 0.6429577464788733
Iemocap - KNN F1: 0.7277911203409533 Acc: 0.7260606060606061
Meld - KNN F1: 0.5947792167273063 Acc: 0.5904214559386973
Iemocap - KNN F1 (macro): 0.5760597935589471
Meld - KNN F1 (macro): 0.4407585720404875

Cosine Similarity between mods: -0.008845211082780864
General - KNN F1: 0.6472219462144913 Acc: 0.6460093896713615
Iemocap - KNN F1: 0.7313628195864195 Acc: 0.7296969696969697
Meld - KNN F1: 0.5962234730821557 Acc: 0.593103448275862
Iemocap - KNN F1 (macro): 0.5782869917527181
Meld - KNN F1 (macro): 0.4393689351519012
'''

## Eval

In [ ]:
#import pickle
#pickle.dump(kmeans, open("./transformer_1_layer_repetindo/kmeans_200_clusters_curr.pkl", 'wb'))

In [ ]:
gc.collect()

In [ ]:
#PATH_TO_SAVE = 'ESTAMOS_PERTO_AMIGO_ESTOU_AQUI_4_freezed_5_layer_pivoting_to_speech_training'

In [ ]:
#torch.load(f'pre_test_final_2/pytorch_model_AudioTextCLIP_epoch_35.bin')['model']
#torch.load(f'{PATH_TO_SAVE}/pytorch_model_AudioTextCLIP_epoch_1.bin')['model']

In [ ]:

PATH_TO_SAVE = 'ESTAMOS_PERTO_AMIGO_ESTOU_AQUI_4_freezed_7'

supcon_model = AudioTextCLIP(
    text_encoder,
    audio_encoder,
    in_features_text=384,
    in_features_audio=dim_embed, 
    hidden_size=1024,
    wide_proj=2048,
    proj_size=128, 
    freeze_text_enc=True,
    freeze_audio_enc=False,
    rate=0.1,
).to(0)
supcon_model.load_state_dict(torch.load(f'{PATH_TO_SAVE}/pytorch_model_AudioTextCLIP_epoch_35.bin')['model'])

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
get_n_params(supcon_model)

In [ ]:
supcon_model.eval()

In [ ]:
import pickle

In [ ]:
torch.norm(test["x_text_wide"][0, :])

In [ ]:
test = supcon_model([["I Hate you, i believe you are shit!", "You are my best friend, love you!"],None, None])
torch.dot(F.normalize(test["x_text_wide"][0, :], dim=-1), F.normalize(test["x_text_wide"][1, :], dim=-1))

In [ ]:
test = supcon_model([["The best man ever, keep the good work!", "you are my best friend, love you!"],None, None])
torch.dot(F.normalize(test["x_text_wide"][0, :], dim=0), F.normalize(test["x_text_wide"][1, :], dim=0))

In [ ]:
test = supcon_model([["I Hate you, i believe you are shit!", "Fuck you, you should not be alive"],None, None])
torch.dot(F.normalize(test["x_text_wide"][0, :], dim=0), F.normalize(test["x_text_wide"][1, :], dim=0))

In [ ]:
with torch.no_grad():
    m, a = audio_tokenizer.batch_tokenize(["./audio/audio_emo/tess.woman.sad.100.wav"])
    audio_input = {
        "features": m.to(0),
        "attn_masks": a.to(0),
    }
    test = supcon_model([["Thanks! you are my best friend, I love you!"],audio_input, None])
    print(torch.dot(F.normalize(test["x_text_wide"][0, :], dim=0), F.normalize(test["x_audio_wide"][0, :], dim=0)))

In [ ]:
with torch.no_grad():
    m, a = audio_tokenizer.batch_tokenize(["./audio/audio_emo/tess.woman.sad.279.wav"])
    audio_input = {
        "features": m.to(0),
        "attn_masks": a.to(0),
    }
    test = supcon_model([["I am sad because my dog died"], audio_input, None])
    print(torch.dot(F.normalize(test["x_text_wide"][0, :], dim=-1), F.normalize(test["x_audio_wide"][0, :], dim=-1)))

In [ ]:
with torch.no_grad():
    m, a = audio_tokenizer.batch_tokenize(["./audio/audio_emo/tess.woman.happy.50.wav"])
    audio_input = {
        "features": m.to(0),
        "attn_masks": a.to(0),
    }
    test = supcon_model([["you are my best friend, love you!"],audio_input, None])
    print(torch.dot(F.normalize(test["x_text_wide"][0, :], dim=0), F.normalize(test["x_audio_wide"][0, :], dim=0)))

In [ ]:
with torch.no_grad():
    m, a = audio_tokenizer.batch_tokenize(["./audio/audio_emo/tess.woman.happy.50.wav", "./audio/audio_emo/crema.man.happy157.wav"])
    audio_input = {
        "features": m.to(0),
        "attn_masks": a.to(0),
    }
    test = supcon_model([["you are my best friend, love you!"],audio_input, None])
    print(torch.dot(F.normalize(test["x_audio_wide"][0, :], dim=0), F.normalize(test["x_audio_wide"][1, :], dim=0)))

In [ ]:
#supcon_model.load_state_dict(torch.load('./pytorch_model_AudioTextCLIPvFinal_epoch_25_only_meld.bin'))

In [ ]:
#supcon_model.audio_encoder.clusterization_model = kmeans

In [ ]:
gc.collect()

In [ ]:
df_train_f =df_train

In [ ]:
#df_dev_audio = pd.concat([df_meld_dev, test_audio], axis=0)

# Param: Select dataset for scoring

In [ ]:
#train_audio_repeated = pd.concat([df_train_audio, df_train_audio,df_train_audio,df_train_audio,df_train_audio,df_train_audio,df_train_audio, df_train_audio,df_train_audio,df_train_audio,df_train_audio,df_train_audio], axis=0).sample(frac=1).reset_index(drop=True)
#test_audio_repeated = pd.concat([df_dev_audio, df_dev_audio,df_dev_audio,df_dev_audio,df_dev_audio,df_dev_audio], axis=0).sample(frac=1).reset_index(drop=True)

train_ds = torch.utils.data.TensorDataset(torch.Tensor(list(range(len(train_df_erc)))))
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=1024, shuffle=False)

test_ds = torch.utils.data.TensorDataset(torch.Tensor(list(range(len(test_df_erc)))))
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1024, shuffle=False)

In [ ]:
gc.collect()

In [ ]:
supcon_model.eval()

proj_val = []
targets_val = []

proj_train = []
targets_train = []

for i, batch_indices in enumerate(tqdm(train_loader, total=len(train_loader))):
    with torch.no_grad():
        batch = train_df_erc.iloc[batch_indices[0]]
        only_text = batch[batch["path"].isna()]
        sentences = only_text["text"].tolist()
        y_text = torch.Tensor(lab_encoder.transform(only_text["label"]))

        only_audio = batch[batch["text"].isna()]
        audio_paths = only_audio["path"].tolist()
        try:
            mfccs, att = audio_tokenizer.batch_tokenize(audio_paths)

            audio_input = {
                "features": mfccs.to(0),
                "attn_masks": att.to(0),
            }
        except:
            audio_input = None

        y_audio = torch.Tensor(lab_encoder.transform(only_audio["label"]))

        mult = batch[batch["text"].notna()]
        mult = mult[mult["path"].notna()]
        mult = mult[mult["label"].notna()]
        y_mult = torch.Tensor(lab_encoder.transform(mult["label"]))

        audio_path_mult = [str(t['path']) for _, t in mult.iterrows()]

        mfccs_mult, att_mult = audio_tokenizer.batch_tokenize(audio_path_mult)

        sentences_mult = [str(t['text']) for _, t in mult.iterrows()]

        multimodal = {'sentences': sentences_mult, 
                      'audio_input': {"features": mfccs_mult.to(0), "attn_masks": att_mult.to(0)}}

        target = torch.cat([y_text, y_audio, y_mult])

        x = [sentences, audio_input, multimodal]

        if len(sentences) == 0:
            x[0] = None
        if len(audio_paths) == 0:
            x[1] = None
        if len(sentences_mult) == 0:
            x[2] = None

        with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16) as autocast, torch.backends.cuda.sdp_kernel(enable_flash=False) as disable:

            out = supcon_model(x)

            # Multimodal loss
            #x_mult_wide = F.normalize(out["x_mult_text_wide"] + out["x_mult_audio_wide"], dim=-1)
            x_mult_wide = F.normalize(out["x_mult_audio_wide"], dim=-1)

        proj_train.append(np.array(x_mult_wide.detach().cpu()))
        targets_train.append(np.array(target.cpu()))

        del x_mult_wide
        gc.collect()
        torch.cuda.empty_cache()
proj_train = np.concatenate(proj_train, axis=0)
targets_train = np.concatenate(targets_train, axis=0)

clf = FaissKNeighbors(k=128)
clf.fit(proj_train, np.array(targets_train, dtype=int))

supcon_model.eval()
preds = []
targets = []
css = 0.0

for i, batch_indices in enumerate(tqdm(test_loader, total=len(test_loader))):
    with torch.no_grad():

        multimodal_batch = test_df_erc.iloc[batch_indices[0]]

        audio_path_mult = [str(t['path']) for _, t in multimodal_batch.iterrows()]
        mfccs_mult, att_mult = audio_tokenizer.batch_tokenize(audio_path_mult)

        sentences_mult = [str(t['text']) for _, t in multimodal_batch.iterrows()]

        multimodal = {'sentences': sentences_mult, 
                      'audio_input': {"features": mfccs_mult.to(0), "attn_masks": att_mult.to(0)}}

        target = torch.Tensor(lab_encoder.transform(list(multimodal_batch["label"])))

        x = [None, None, multimodal]
        with torch.cuda.amp.autocast(enabled=True, dtype=torch.float16) as autocast, torch.backends.cuda.sdp_kernel(enable_flash=False) as disable:
            out = supcon_model(x)

            # Multimodal loss
            #out_x_wide = F.normalize(out["x_mult_text_wide"] + out["x_mult_audio_wide"], dim=-1)
            out_x_wide = F.normalize(out["x_mult_audio_wide"], dim=-1)

        cs = F.cosine_similarity(out["x_mult_text_wide"], out["x_mult_audio_wide"])

        wide = np.array(out_x_wide.cpu())
        pred = clf.predict(wide)
        preds.append(pred)

        assert len(wide) == len(pred)

        proj_val.append(wide)
        targets_val.append(np.array(target.cpu()))
        css += np.sum(np.array(cs.cpu()))
        del out_x_wide
        gc.collect()
        torch.cuda.empty_cache()

proj_val = np.concatenate(proj_val, axis=0)
targets_val = np.concatenate(targets_val, axis=0)

preds = np.array(np.concatenate(preds, axis=0))

css = css / len(test_df_erc)

general_f1 = f1_score(targets_val, preds, average='weighted')
general_acc = accuracy_score(targets_val, preds)

print(f'Cosine Similarity between mods: {css}')

meld_idx = test_df_erc[test_df_erc["source"] == "meld"].index
iemocap_idx = test_df_erc[test_df_erc["source"] != "meld"].index

general_f1_iemocap = f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='weighted')
general_acc_iemocap = accuracy_score(targets_val[iemocap_idx], preds[iemocap_idx])

general_f1_meld = f1_score(targets_val[meld_idx], preds[meld_idx], average='weighted')
general_acc_meld = accuracy_score(targets_val[meld_idx], preds[meld_idx])

print(f'General - KNN F1: {general_f1} Acc: {general_acc}')
print(f'Iemocap - KNN F1: {general_f1_iemocap} Acc: {general_acc_iemocap}')
print(f'Meld - KNN F1: {general_f1_meld} Acc: {general_acc_meld}')

print(f"Iemocap - KNN F1 (macro): {f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='macro')}")
print(f"Meld - KNN F1 (macro): {f1_score(targets_val[meld_idx], preds[meld_idx], average='macro')}")

tsne = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=5).fit_transform(proj_val)

sns.scatterplot(x=tsne[:, 0], y=tsne[:, 1], hue=lab_encoder.inverse_transform(list(np.array(targets_val, dtype=int))) , palette='tab10')
plt.show()

In [ ]:
train_df_erc

In [ ]:
meld_train_idx = train_df_erc[train_df_erc["path"].apply(lambda x: True if "MELD" in x else False)].index

In [ ]:
iemocap_train_idx = train_df_erc[train_df_erc["path"].apply(lambda x: False if "MELD" in x else True)].index

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
mean_test = proj_train[meld_train_idx].mean(axis=0)
std_test = proj_train[meld_train_idx].std(axis=0)
clf = FaissKNeighbors(k=128)
clf.fit((proj_train[meld_train_idx]-mean_test)/std_test, np.array(targets_train[meld_train_idx], dtype=int))

preds = clf.predict((proj_val-mean_test)/std_test)

general_f1_meld = f1_score(targets_val[meld_idx], preds[meld_idx], average='weighted')
general_acc_meld = accuracy_score(targets_val[meld_idx], preds[meld_idx])

from sklearn.metrics import classification_report

print(classification_report(targets_val[meld_idx], preds[meld_idx], digits=4))

In [ ]:
len(targets_val[iemocap_idx])

In [ ]:
set(lab_encoder.inverse_transform(np.array(targets_val[iemocap_idx], dtype=int)))


In [ ]:
df_iemocap_orig = pd.read_json("emotions.json").reset_index(drop=False)
df_iemocap_orig = pd.melt(df_iemocap_orig, id_vars=['index'], value_vars=['train', 'val', 'test']).dropna().drop(columns=["variable"]).rename(columns={"index":"id", "value": "orig_label"}).reset_index(drop=True)
df_iemocap_orig = df_iemocap_orig[df_iemocap_orig["orig_label"].notna() & (df_iemocap_orig["orig_label"] != "undecided")].reset_index(drop=True)
df_iemocap_orig

In [ ]:
def cleaning_shit(x):
    if "MELD" in x:
        return None
    x = x.replace("val/", "")
    x = x.replace("train/", "")
    x = x.replace("test/", "")
    l = len("/home/vmachado/Documents/multimodal-datasets/IEMOCAP/raw-audios/")
    return x[l:].replace(".wav", "")


In [ ]:
#train_df_erc_iemocap = train_df_erc[train_df_erc["path"].apply(lambda x: True if "IEMOCAP" in x else False)]
train_df_erc["id"] = train_df_erc["path"].apply(cleaning_shit)
train_df_erc_iemocap = train_df_erc.dropna()
train_df_erc_iemocap = train_df_erc_iemocap.merge(df_iemocap_orig, on="id", how="inner").dropna()
train_df_erc_iemocap

In [ ]:
len(iemocap_train_idx)

In [ ]:
test_df_erc["id"] = test_df_erc["path"].apply(lambda x: x[len('/home/vmachado/Documents/multimodal-datasets/IEMOCAP/raw-audios/test/'):].replace(".wav", "")) 

In [ ]:
test_df_erc_iemocap = test_df_erc[test_df_erc["source"] == "iemocap"].reset_index(drop=True)
test_df_erc_iemocap

In [ ]:
test_df_erc_iemocap = test_df_erc_iemocap.merge(df_iemocap_orig, on="id", how="inner")
test_df_erc_iemocap

In [ ]:
new_lab = LabelEncoder().fit(train_df_erc_iemocap["orig_label"])

In [ ]:
train_df_erc_iemocap["orig_label"].unique()

In [ ]:
test_df_erc_iemocap["orig_label"].unique()

In [ ]:
train_df_erc_iemocap

In [ ]:
test_df_erc_iemocap

In [ ]:
correct_labels_train = new_lab.transform(train_df_erc_iemocap["orig_label"])
correct_labels_test = new_lab.transform(test_df_erc_iemocap["orig_label"])

In [ ]:
correct_labels_train

In [ ]:
lab_encoder.classes_

In [ ]:
test_df_erc[test_df_erc["source"] == "iemocap"]["label"].value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
mean_test = proj_train[iemocap_train_idx].mean(axis=0)
std_test = proj_train[iemocap_train_idx].std(axis=0)
clf = FaissKNeighbors(k=128)
clf.fit((proj_train[iemocap_train_idx]-mean_test)/std_test, np.array(targets_train[iemocap_train_idx], dtype=int))

preds = clf.predict((proj_val-mean_test)/std_test)

from sklearn.metrics import classification_report

print(classification_report(targets_val[iemocap_idx], preds[iemocap_idx], digits=4))

In [ ]:
general_f1_iemocap = f1_score(targets_val[iemocap_idx], preds[iemocap_idx], average='weighted')
general_acc_iemocap = accuracy_score(targets_val[iemocap_idx], preds[iemocap_idx])



In [ ]:
general_f1_iemocap

In [ ]:
general_f1_meld

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(targets_val[iemocap_idx], list(map(lambda x: x if x != 1 else 6, preds[iemocap_idx])), digits=4))

## 